# Effects of Lock Contention

This notebook visualizes the performance difference between a shared-file write workload that was performed with writes aligned on specific boundaries (e.g., every write starts on a 1 MiB offset) and writes performed without any alignment.

The specific data being plotted came from a set of IOR runs that used the HDF5 API.  Because HDF5 includes small metadata headers before each dataset, it is prone to performing slightly misaligned I/Os unless `H5Pset_alignment()` is explicitly used.  Each IOR test was performed ten times, and different write sizes were chosen to demonstrate how smaller I/Os (and therefore more of them) result in more time spent on lock contention.

In [ ]:
%matplotlib inline

In [ ]:
import glob
import pandas
import matplotlib.pyplot
matplotlib.rcParams['font.size'] = 18

In [ ]:
import iorparse

In [ ]:
# Load results
raw_results = {}
for pattern in 'aligned', 'unaligned':
    raw_results[pattern] = []
    for ior_output in glob.glob('results/%s_*.out' % pattern):
        raw_results[pattern] += iorparse.parse(open(ior_output, 'r'), access='write')

In [ ]:
# Collate results
results = {}
columns = set([])
for pattern in 'aligned', 'unaligned':
    results[pattern] = {}
    for result in raw_results[pattern]:
        key = result['xfersize_kib']
        if result['xfersize_kib'] >= 1024:
            key = "%d MiB" % (key / 1024)
        else:
            key = "%d KiB" % key
        val = result['bw_mibs']
        if key not in results[pattern]:
            results[pattern][key] = []
        results[pattern][key].append(val)
        # Track both the co
        columns.add((result['xfersize_kib'], key))

In [ ]:
# Build dataframes of the data for ease of plotting
sorted_columns = [x[1] for x in sorted(list(columns))]

avg_rates = pandas.DataFrame([
        pandas.DataFrame.from_dict(results['unaligned']).mean(),
        pandas.DataFrame.from_dict(results['aligned']).mean(),
    ],
    index=["Unaligned", "Aligned, 1 MiB"])[sorted_columns].T
stdev_rates = pandas.DataFrame([
        pandas.DataFrame.from_dict(results['unaligned']).std(),
        pandas.DataFrame.from_dict(results['aligned']).std(),
    ],
    index=["Unaligned", "Aligned, 1 MiB"])[sorted_columns].T

In [ ]:
print(avg_rates)
print(stdev_rates)

In [ ]:
error_kw = {
    'capsize': 10,
    'ecolor': '#00000088',
    'capthick': 2,
}

fig, ax = matplotlib.pyplot.subplots(figsize=(8,6))

# Error bars are two standard deviations
errorbars = 1.0 * stdev_rates

avg_rates.plot.bar(width=0.80, ax=ax, yerr=errorbars, error_kw=error_kw)
ax.grid(axis='y')
ax.set_ylabel("Write Rate (MiB/s)")
ax.set_xlabel("I/O Size")
ax.set_axisbelow(True)
ax.set_xticklabels(labels=ax.get_xticklabels(), rotation=0)

The performance difference is smallest for 4 MiB write transactions because there simply weren't that many write operations issued.  Each time a 4 MiB write was issued, there would be lock negotiation, but the cost of that lock waiting was amortized by the time spent actually writing out the 4 MiB chunk after the lock had been obtained.

In [ ]:
ax.get_figure().savefig('unaligned-access-performance.pdf', bbox_inches='tight')